In [2]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '2'
import sys
from pathlib import Path
import anndata as ad
import scanpy as sc
import numpy as np
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import seaborn as sns
from matplotlib import pyplot as plt

sys.path.insert(0, "../../scripts/methods/")
from my_slat import slat_align, slat_align_ref
sys.path.insert(0, "/home/ylu/project")
from utils import *
import time


%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
data_folder = "./results/split_slices/"
results_folder = "./results/SLAT/"
figures_folder = "./results/figures/SLAT"
Path(results_folder).mkdir(parents=True, exist_ok=True)

In [4]:
## load the data
from tqdm import tqdm
slices = []
for i in tqdm(range(59)):
    slices.append(ad.read_h5ad(os.path.join(data_folder, f"slice_{i}.h5ad")))

100%|█████████████████| 59/59 [02:57<00:00,  3.01s/it]


In [5]:
spatial_key = "spatial"
anno_key = "class"

In [6]:
## Rotate the data
rotate_key = "spatial_rot"
rotations = np.load("./results/random_rotations.npy", allow_pickle=True)
for i in range(len(slices)):
    slices[i].obsm[rotate_key] = slices[i].obsm[spatial_key][:,:2].copy()
    mean = np.mean(slices[i].obsm[rotate_key], axis=0)
    slices[i].obsm[rotate_key] = slices[i].obsm[rotate_key] - mean
    slices[i].obsm[rotate_key] = slices[i].obsm[rotate_key] @ rotations[i].T + mean
    

In [7]:
spatial_key = "spatial_rot"
key_added = "aligned_spatial"

In [8]:
## Perform the SLAT alignment
sampling_num = 20000
for i in tqdm(range(len(slices)-1)):
    slice1, slice2 = slices[i].copy(), slices[i+1].copy()
    sampline_idx1 = np.random.choice(slice1.shape[0], sampling_num, replace=False) if slice1.shape[0] > sampling_num else np.arange(slice1.shape[0])
    sampline_idx2 = np.random.choice(slice2.shape[0], sampling_num, replace=False) if slice2.shape[0] > sampling_num else np.arange(slice2.shape[0])
    slice1 = slice1[sampline_idx1,:]
    slice2 = slice2[sampline_idx2,:]
    time_start = time.time()
    align_slices, pis = slat_align(
        models = [slice1, slice2],
        spatial_key=spatial_key,
        key_added=key_added,
    )
    time_end = time.time()
    matches = pis
    R, t=solve_RT_by_correspondence(align_slices[1].obsm[key_added], align_slices[1].obsm[spatial_key])
    alignment_results = {'sampling_idx1': sampline_idx1, 'sampline_idx2': sampline_idx2, 'R': R, 't': t, 'matches': matches, 'time': time_end - time_start}
    np.save(os.path.join(results_folder, f"slice_{i}_{i+1}_sampling_{sampling_num}.npy"), alignment_results, allow_pickle=True)

  0%|                          | 0/58 [00:00<?, ?it/s]

Calculating spatial neighbor graph ...
The graph contains 226052 edges, 20000 cells.
11.3026 neighbors per cell on average.
Calculating spatial neighbor graph ...
The graph contains 227076 edges, 20000 cells.
11.3538 neighbors per cell on average.
Use DPCA feature to format graph


/home/ylu/anaconda3/envs/scSLAT/lib/python3.8/site-packages/scanpy/preprocessing/_normalization.py:169: UserWarning: Received a view of an AnnData. Making a copy.
  view_to_actual(adata)
/home/ylu/anaconda3/envs/scSLAT/lib/python3.8/site-packages/scanpy/preprocessing/_simple.py:843: UserWarning: Received a view of an AnnData. Making a copy.
  view_to_actual(adata)
/home/ylu/anaconda3/envs/scSLAT/lib/python3.8/site-packages/scanpy/preprocessing/_simple.py:843: UserWarning: Received a view of an AnnData. Making a copy.
  view_to_actual(adata)


Warning! Dual PCA is using GPU, which may lead to OUT OF GPU MEMORY in big dataset!
Choose GPU:0 as device
Running
---------- epochs: 1 ----------
---------- epochs: 2 ----------
---------- epochs: 3 ----------
---------- epochs: 4 ----------
---------- epochs: 5 ----------
---------- epochs: 6 ----------
Training model time: 2.56


  2%|▎                 | 1/58 [00:22<21:04, 22.19s/it]

...

 98%|████████████████▋| 57/58 [27:33<00:33, 33.47s/it]

Calculating spatial neighbor graph ...
The graph contains 226688 edges, 20000 cells.
11.3344 neighbors per cell on average.
Calculating spatial neighbor graph ...
The graph contains 159023 edges, 13963 cells.
11.388884910119602 neighbors per cell on average.
Use DPCA feature to format graph


/home/ylu/anaconda3/envs/scSLAT/lib/python3.8/site-packages/scanpy/preprocessing/_normalization.py:169: UserWarning: Received a view of an AnnData. Making a copy.
  view_to_actual(adata)
/home/ylu/anaconda3/envs/scSLAT/lib/python3.8/site-packages/scanpy/preprocessing/_simple.py:843: UserWarning: Received a view of an AnnData. Making a copy.
  view_to_actual(adata)
/home/ylu/anaconda3/envs/scSLAT/lib/python3.8/site-packages/scanpy/preprocessing/_simple.py:843: UserWarning: Received a view of an AnnData. Making a copy.
  view_to_actual(adata)


Warning! Dual PCA is using GPU, which may lead to OUT OF GPU MEMORY in big dataset!
Choose GPU:0 as device
Running
---------- epochs: 1 ----------
---------- epochs: 2 ----------
---------- epochs: 3 ----------
---------- epochs: 4 ----------
---------- epochs: 5 ----------
---------- epochs: 6 ----------
Training model time: 2.64


100%|█████████████████| 58/58 [28:00<00:00, 28.98s/it]
